<span style="font-size:24pt; color:black">Notebook for creating AR6 style p-box data</span>

<ul style="font-size:16pt; color:black"> 
Make sure to set the <b>expFolder</b> to point to your facts exp output folder. This notebook creates the following folders within the current directory. 
<li></li>
<li> 1_workflow  </li>
<li> 2_workflow_quantiles </li>
<li> 3_pbox </li>
<li> 4_confidence_level_files </li>
</ul>


In [2]:
import shutil
import os
PD=os.getcwd()
import fun_pbox_Gen as fn
#
ssp=["ssp119","ssp126","ssp245","ssp370","ssp585"]
#
workflow = ["wf_1e","wf_1f","wf_2e","wf_2f","wf_3e", "wf_3f", "wf_4"]
#
expFolder="/projects/kopp/facts-experiments/221217/"

---
## <font color=red> Generate:: 1_workflow  </font>
---

In [3]:
Workflow_path = fn.FOcr8("1_workflow")
for indv_workflow0,indv_workflow1 in enumerate(workflow):
    wf_path=fn.FOcr8_1(Workflow_path, indv_workflow1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_workflow1 in ['wf_3e', 'wf_3f'] and ssp1 in['ssp119','ssp370']: continue
        if indv_workflow1 in ['wf_4'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        ssp_path=fn.FOcr8_1(wf_path, ssp1) #create ssp folders within the wf
        #
        # for wfcomp0,wfcomp1 in enumerate(eval(f"{indv_workflow1}")):
        for wfcomp0,wfcomp1 in enumerate(fn.component[indv_workflow1]):
            sourcefile_path=expFolder+"coupling."+ssp1+"/output"
            shutil.copy2(sourcefile_path+'/coupling.'+ssp1+'.'+wfcomp1, ssp_path) # copy the file to the ssp folder in the workflow

        for wfcomp0,wfcomp1 in enumerate(fn.total[indv_workflow1]):
            sourcefile_path=expFolder+"coupling."+ssp1+"/output"
            shutil.copy2(sourcefile_path+'/coupling.'+ssp1+'.'+wfcomp1, ssp_path) # copy the file to the ssp folder in the workflow

---
## <font color=red> Generate:: 2_workflow_quantiles  </font>
---

In [4]:
# Create a folder
Workflow_path = fn.FOcr8("2_workflow_quantiles")
#
for indv_workflow0,indv_workflow1 in enumerate(workflow):
    wf_path=fn.FOcr8_1(Workflow_path, indv_workflow1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_workflow1 in ['wf_3e', 'wf_3f'] and ssp1 in['ssp119','ssp370']: continue
        if indv_workflow1 in ['wf_4'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        # Component
        ssp_path=fn.FOcr8_1(wf_path, ssp1) #create ssp folders within the wf
        for wfcomp0,wfcomp1 in enumerate(fn.component[indv_workflow1]):
            #Convert to dist calling the fn directly
            inFile = PD+'/1_workflow/'+indv_workflow1+'/'+ssp1+'/coupling.'+ssp1+'.'+wfcomp1
            outFile = ssp_path+'/coupling.'+ssp1+'.'+wfcomp1
            fn.ConvertSamplesToDist(inFile, outFile)
        # Total
        for wfcomp0,wfcomp1 in enumerate(fn.total[indv_workflow1]):
            #Convert to dist calling the fn directly
            inFile = PD+'/1_workflow/'+indv_workflow1+'/'+ssp1+'/coupling.'+ssp1+'.'+wfcomp1
            outFile = ssp_path+'/coupling.'+ssp1+'.'+wfcomp1
            fn.ConvertSamplesToDist(inFile, outFile)

---
## <font color=red> Generate:: 003_pbox  </font>
---

In [6]:
Pbox = ["pb_1e","pb_1f","pb_2e","pb_2f"]
#
# Create a folder
Pbox_path = fn.FOcr8("3_pbox")
#
for indv_Pbox0,indv_Pbox1 in enumerate(Pbox):
    pb_path=fn.FOcr8_1(Pbox_path, indv_Pbox1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_Pbox1 in ['pb_2e', 'pb_2f'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        ssp_path=fn.FOcr8_1(pb_path, ssp1) #create ssp folders within the pb
        #
        if indv_Pbox1 in ['pb_1e','pb_1f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # larmipAIS (common to pb_1e & pb_1f)
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*larmip.AIS*")
            #
            if indv_Pbox1 in ['pb_1e']:
                # AIS Pbox component
                fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1e");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

                #
            if indv_Pbox1 in ['pb_1f']:
                # AIS Pbox component.
                fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1f");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")


        
        if indv_Pbox1 in ['pb_2e']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # emulandice.AIS
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*emulandice.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.glaciers*");
            
            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # Glaciers Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.glaciers")
            fname2=fn.fileNAME0(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"glaciers-pb"+indv_Pbox1.split('_')[1]+"-glaciers-"+ssp1+"_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  
            
            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2e");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1e"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3e"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        if indv_Pbox1 in ['pb_2f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.icesheets_AIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");


            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  

            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2f");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1f"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3f"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

---
## <font color=red> Generate:: 006_CI-level_OP_files  </font>
---

In [8]:
pboxdir=PD+'/3_pbox'
outdir = fn.FOcr8("4_confidence_level_files")
fn.GenerateConfidenceFiles(pboxdir, outdir)